
<div style="border:1px solid black; padding:20px 20px;text-align: justify;text-justify: inter-word">
    <strong>Exercise Session 6 - Odometry and Localisation<br/> Autumn 2020 <br/> Duration : 4 hours (2 in session + 2 at home)</strong><br/><br/>
    <span style="text-decoration:underline;font-weight:bold;">How to use this notebook?</span><br/>
    This notebook is made of text cells and code cells. The code cells have to be <strong>executed</strong> to see the result of the program. To execute a cell, simply select it and click on the "play" button (<span style="font: bold 12px/30px Arial, serif;">&#9658;</span>) in the tool bar just above the notebook, or type <code>shift + enter</code>. It is important to execute the code cells in their order of appearance in the notebook.<br/>
You can make use of the table of contents to navigate easily between sections.
</div>

<br/>

<div style="justify;text-justify: inter-word">
So that you may familiarise with the notebooks and the basic python syntax, the exercises are provided in notebook form and whenever there are any calculations to be made, we encourage you to do them by code. Also, if you want to take notes, we encourage you to use the markdown or Raw NBConvert cells. 
</div>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Learning-Goals" data-toc-modified-id="Learning-Goals-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Learning Goals</a></span></li><li><span><a href="#Requirements" data-toc-modified-id="Requirements-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Requirements</a></span></li><li><span><a href="#Odometry" data-toc-modified-id="Odometry-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Odometry</a></span><ul class="toc-item"><li><span><a href="#Introduction-and-goal-of-odometry-exercise" data-toc-modified-id="Introduction-and-goal-of-odometry-exercise-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Introduction and goal of odometry exercise</a></span></li><li><span><a href="#Move-and-turn" data-toc-modified-id="Move-and-turn-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Move and turn</a></span></li></ul></li><li><span><a href="#Localisation" data-toc-modified-id="Localisation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Localisation</a></span><ul class="toc-item"><li><span><a href="#Computing-the-probabilities" data-toc-modified-id="Computing-the-probabilities-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Computing the probabilities</a></span></li><li><span><a href="#Implementation-on-Thymio" data-toc-modified-id="Implementation-on-Thymio-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Implementation on Thymio</a></span></li></ul></li></ul></div>

# Learning Goals


- Program Thymio to observe odometry errors. 


- Compute a probabilistic localisation with sensing and motion uncertainties. 


- Program a simple localisation algorithm that takes into account sensing and motion uncertainties.


# Requirements

- Thymio

- The A3 sheet with the stripe pattern (localization-map.pdf)

![localisation map](images/localization-map.png)


# Odometry

## Introduction and goal of odometry exercise

Odometry is an essential component of robot localisation, but is subject to several problems, linkingtheory and practice. The goal of this exercise is to observe several of these problems by implementinga very simple trajectory using the Thymio robot.

## Move and turn


Implement on Thymio a program that makes it continuously turn and move forward:


    1. Turn 360 + 180 degrees


    2. Move forward a distance corresponding to the movement the wheel did during the turning step.


Of course this movement needs to be carefully calibrated for your specific robot. Therefore first plan and implement a calibration procedure.


Place a pencil in the central hole of Thymio and draw the trajectory. Please note that this will require two A4 sheets of paper. 


Observe the result, iterate to improve the result (do you need other calibrations?) and comment the various errors you correct.


<blockquote>
    
<span style="color: #2980B9 ;">


To calibrate the odometry in rotation (the only needed here) based on time and speed (the only possibility here, having no encoders) it is useful to have a program that allows to set a speed and count time. The following program allows to start and stop rotating, enabling to count how much time (in 10ms units) is taken to make 10 revolutions. 


    var counting = 0
    var time = 0

    timer.period[0] = 10 # milliseconds

    onevent button.right # start counting
      counting = 1
      time = 0
      motor.left.target = 100
      motor.right.target = -100

    onevent button.center # stop counting
      counting = 0
      motor.left.target = 0
      motor.right.target = 0

    onevent timer0
      if counting == 1 then
        time = time + 1
      end


</span> 
</blockquote>

<blockquote>
    
<span style="color: #2980B9 ;">


Once you know the time required to make one revolution of the robot (in this case on itself at the given speed 100,-100), you need to write a code allowing to move and turn in repetition:


    const FORWARD = 1
    const TURN = 2
    const ONETURN = 8873
    const SAMPLING = 5

    var state = 0
    var time = 0

    timer.period[0] = SAMPLING # 5 ms resolution

    onevent button.forward
      state = FORWARD
      time = 0
      motor.left.target = 100
      motor.right.target = 100

    onevent button.center
      state = 0
      time = 0
      motor.left.target = 0
      motor.right.target = 0


    onevent timer0
      time = time + 1
      if state == FORWARD then
        if time > ONETURN*3/SAMPLING/2 then
          time = 0
          state = TURN
          motor.left.target = 100
          motor.right.target = -100
        end
      end
      if state == TURN then
        if time > ONETURN*3/SAMPLING/2 then
          time = 0
          state = FORWARD
          motor.left.target = 100
          motor.right.target = 99
        end
      end


The figure below shows an example of several runs. Several observations can be done:


![Problems](Images/variousproblems.jpg)
*Example with bad calibration and non-straight motion.*


+ The robot does not go straight, this is a problem of calibration of the two speeds. This has been corrected in the program above by giving different speeds to the two wheels.

+ In turn 5 the robot slipped, turning less than required.

+ The calibration for rotation seems correct. If the calibration is not correct and the robot goes staight, the global shape is very similar.



The Figure below shows a much better run. In this case both the turning calibration and the straight motion are good, and we see on the sides that the dispersion of the turning points is much larger vertically than horizontally, showing the effect of the errors of odometry seen in the course.



![Noise Only](Images/noiseonly.jpg)
*Example with good calibration and straight motion.*

</span> 
</blockquote>

# Localisation

The goal of this second part is to practice the localisation as explained during the course.

## Computing the probabilities


If you do not feel confident with the explanation given during the course, do again all computations, for instance in a spreadsheet.


Consider the map below with the initial position of the robot, an initial probability distribution, and its movement restricted to moving from left to right. In the motion model, there is an error of motion with a probability of $10\%$ of incorrect motion to the neighbour case, and therefore $80\%$ or correct displacement. For the perception model, consider a probability of wrong measurement of $0.25$, meaning that if the ground is dark there is a probability of $25\%$ of reading a light color and viceversa.


Compute using a Bayes filter (also called Markov localisation):


- The prediction probabilities and measurement update based on the given movement (one step), assuming a correct movement and measurement. Give the final probability distribution after this step.


Complete as many rows as needed, indicating concisely what is performed on the left.

![Localisation](Images/localization.png)


| Map                        |       | X     |       |       | X     | X       |       |       |       |      |
|----------------------------|-------|-------|-------|-------|-------|---------|-------|-------|-------|------|
| Pos                        |  0    |  0    | 0     |  0    |  0    |   r     |  0    |  0    |  0    |  0   |
| Initial state.             |  0    |  0    | 0     |  0    |  0    |   1     |  0    |  0    |  0    |  0   |
| After move                 |  0    |  0    | 0     |  0    |  0    |  0.1    | 0.8   | 0.1   |  0    |  0   |
| Meas. prob. (no wall)      |  0.75 |  0.25 | 0.75  | 0.75  | 0.25  |  0.25   | 0.75  | 0.75  | 0.75  | 0.75 |
| Prod.                      |  0    |  0    | 0     |  0    |  0    |  0.025  | 0.6   | 0.075 |  0    |  0   |
| Normalized (/0.7)  |  0    |  0    | 0     |  0    |  0    | 0.0357 | 0.857 |0.107|  0    |  0    |

## Implementation on Thymio

Implement the localisation algorithm on the Thymio robot. You can simplify the program by moving yourself the robot from one position to the other and pressing a button when you are ready to make the measurement, simplifying the motion control of the robot and focusing on the computation of the probability. Use the same example as given in the course, based on a set of grey levels on the ground. The corresponding A3 ground is provided. 

    
<span style="color: #2980B9 ;">


>  The code is provided in the folder AESL solutions 
    
![Map](Images/map-robot.png)